# Telecom Service Assurance AI Model (Simple-NN) for NPS Insights
Author: Fatih E. NAR

## Introduction
In this notebook, we showcase a machine learning model to create NPS predictions for telecom networks. Net Promoter Score (NPS) is a customer loyalty metric that measures customers' willingness to recommend a company's products or services to others. It is widely used for assessing customer satisfaction and predicting business growth.

Our prediction for the Net Promoter Score (NPS) aims to estimate how likely customers are to recommend our product or service based on various features or factors. By building a predictive model, we can forecast NPS scores for new or unseen data, which can help us understand and improve customer satisfaction.

In [ ]:
# Install the required packages
%pip install -r requirements.txt

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Check if any accelerator is available 
if torch.cuda.is_available():
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    device = torch.device("cuda")
    torch.cuda.empty_cache()
    max_memory_mb = 11.9 * 1024 #Set for <12GB on RTX 4070 Super or <20GB for A4500
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = f'max_split_size_mb:{max_memory_mb}'
    print(torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    #os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Assuming GPU 1 is the A4500
# Check if MPS (Apple Silicon GPU) is available
elif torch.backends.mps.is_available():
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Load the generated data
data = pd.read_csv('data/servass_data.csv.xz', compression='xz', parse_dates=['timestamp'])

# Assuming 'nps' is a column in your data and other columns are features
features = ['latency', 'jitter', 'packet_loss', 'throughput', 'cpu_usage', 'memory_usage', 'port_speed', 'traffic_volume']
target = 'nps'

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(device)
y_train = torch.tensor(train_data[target].values, dtype=torch.float32).view(-1, 1).to(device)
X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(device)
y_val = torch.tensor(val_data[target].values, dtype=torch.float32).view(-1, 1).to(device)


In [ ]:
# Define a simple neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 128)
        self.output_layer = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output_layer(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN(input_dim=len(features)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_val).flatten().cpu()
    labels = y_val.flatten().cpu()
    predictions_np = predictions.numpy()
    labels_np = labels.numpy()
    mse = mean_squared_error(labels_np, predictions_np)
    mae = mean_absolute_error(labels_np, predictions_np)
    mape = np.mean(np.abs((labels_np - predictions_np) / labels_np)) * 100
    print(f"Validation MSE: {mse:.4f}")
    print(f"Validation MAE: {mae:.4f}")
    print(f"Validation MAPE: {mape:.2f}%")

In [ ]:
# Plot predictions vs actual values with color coding for errors
errors = np.abs(labels_np - predictions_np)
plt.figure(figsize=(10, 6))
scatter = plt.scatter(labels_np, predictions_np, c=errors, cmap='coolwarm', alpha=0.5)
# Ideal line for perfect predictions
plt.plot([0, 10], [0, 10], '--', color='red', label='Ideal 1:1 Line')
plt.xlabel('Actual NPS')
plt.ylabel('Predicted NPS')
plt.title('Actual vs Predicted NPS')
colorbar = plt.colorbar(scatter)
colorbar.set_label('Prediction Error')
plt.text(10, -2, 'Blue = Good (Small Error)\nRed = Bad (Large Error)', fontsize=8, bbox=dict(facecolor='white', alpha=0.5))
plt.legend()
plt.show()